In [1]:
import os
import glob
import numpy as np
from torchvision import transforms
import torch
from PIL import Image
import pickle
from facenet_pytorch import InceptionResnetV1

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Thư mục dataset
dataset_folder = "faces/"

# Hàm chuyển đổi hình ảnh sang embedding sử dụng FaceNet
def img_to_encoding(img_path, model):
    img = Image.open(img_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        encoding = model(img_tensor).cpu().numpy()
    return encoding

# Hàm khởi tạo cơ sở dữ liệu với embeddings của FaceNet
def initialize(model):
    database = {}

    # Tải tất cả hình ảnh của các cá nhân để nhận diện vào cơ sở dữ liệu
    for folder in glob.glob(os.path.join(dataset_folder, "*")):
        if os.path.isdir(folder):
            identity = os.path.basename(folder)
            encodings = []
            for img_path in glob.glob(os.path.join(folder, '*')):
                try:
                    enc = img_to_encoding(img_path, model)
                    encodings.append(enc)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
            if encodings:
                database[identity] = np.mean(encodings, axis=0)

    return database

# Lưu cơ sở dữ liệu vào một file
database = initialize(facenet_model)
with open('face_database.pkl', 'wb') as f:
    pickle.dump(database, f)
print("Database saved successfully.")

Database saved successfully.
